# Pytorch - Redes Neuronales

In [ ]:
import torch
import numpy as np

In [ ]:
#dimensiones de las capas de la red neuronal
#D_in es la dimensión de la capa de entrada
#H es la dimensión de la capa oculta, esta es la cantidad de neuronas en la capa oculta de la red
#D_out es la dimensión de la capa de salida y se establece en 10. Esto indica que la red neuronal produce un vector de salida de 10 dimensiones, lo que podría ser apropiado para una tarea de clasificación en la que hay 10 clases posibles.

D_in, H, D_out = 784, 55, 10

#Se crea un modelo de red neurona, donde las capas se aplican en orden uno tras otro

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H), #Esta es la capa lineal que conecta la capa de entrada con la capa oculta.
    torch.nn.ReLU(),          # Esta es una función de activación ReLU (Rectified Linear Unit) Ayuda a introducir no linealidad en la red neuronal.
    torch.nn.Linear(H, D_out),#Esta es la capa lineal que conecta la capa oculta con la capa de salida
)

In [ ]:
#Se pasa un tensor de datos de entrada aleatorios a través de tu modelo de red neuronal
outputs = model(torch.randn(64, 784))

outputs.shape

torch.Size([64, 10])

In [ ]:
#se utiliza para mover el modelo de red neuronal y sus parámetros a la GPU
model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=55, bias=True)
  (1): ReLU()
  (2): Linear(in_features=55, out_features=10, bias=True)
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Se carga el dataset
data = np.loadtxt("/content/drive/MyDrive/Inteligencia Artificial/InteligenciaArtificialMACG/Tareas/datasets/train-Dig-MNIST.csv", delimiter=',',skiprows=1)
# print(data)
X, Y = data[:, 1:], data[:, 0]
print(X.shape)
print(Y.shape)

(60000, 784)
(60000,)


In [ ]:

# normalización y split
#astype(int) se usa para convertir las etiquetas en enteros.
X_train, X_test, y_train, y_test = X[:50000] / 255., X[50000:] / 255., Y[:50000].astype(np.int), Y[50000:].astype(np.int)

<ipython-input-92-c0dcc31a0f42>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train, X_test, y_train, y_test = X[:50000] / 255., X[50000:] / 255., Y[:50000].astype(np.int), Y[50000:].astype(np.int)


In [ ]:
# función de pérdida y derivada

# Softmax toma un tensor x como entrada y calcula la exponencial de cada elemento en el tensor.
#  Luego, suma estas exponenciales a lo largo del último eje y divide cada exponencial por esta suma para obtener una distribución de probabilidad.
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

#La función calcula la pérdida de entropía cruzada para cada muestra, que penaliza las diferencias entre las predicciones y las etiquetas reales.
#Luego, toma la media de estas pérdidas individuales para obtener la pérdida total.
def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [ ]:
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# mueve todo el modelo y sus parámetros a la GPU
model.cuda()

Sequential(
  (0): Linear(in_features=784, out_features=55, bias=True)
  (1): ReLU()
  (2): Linear(in_features=55, out_features=10, bias=True)
)

In [ ]:
# convertimos datos a tensores y copiamos en gpu

X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

# bucle entrenamiento
# número total de épocas que se utilizarán para entrenar un modelo
epochs = 100

# tasa de aprendizaje (learning rate) que se utilizará durante el entrenamiento.
lr = 0.8

# Esta variable indica con qué frecuencia se registrarán métricas o información durante el entrenamiento
log_each = 10

# Inicializamos una lista vacia
l = []

#Este bucle itera a través de las épocas de entrenamiento.
for e in range(1, epochs + 1):

    # forward
    #produce predicciones y_pred para las muestras en X_t
    y_pred = model(X_t)

    # loss
    #Calculas la pérdida utilizando la función de pérdida de entropía cruzada
    loss = cross_entropy(y_pred, Y_t)
    #Agrega el valor de la pérdida a la lista para llevar un registro de cómo cambia la pérdida a lo largo de las épocas.
    l.append(loss.item())

    # ponemos a cero los gradientes
    model.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad
  # imprime el número de época actual y el valor promedio de la pérdida en ese punto
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 10/100 Loss 1.66202
Epoch 20/100 Loss 1.11478
Epoch 30/100 Loss 0.87515
Epoch 40/100 Loss 0.72087
Epoch 50/100 Loss 0.62033
Epoch 60/100 Loss 0.54999
Epoch 70/100 Loss 0.49781
Epoch 80/100 Loss 0.45740
Epoch 90/100 Loss 0.42507
Epoch 100/100 Loss 0.39854


In [ ]:
from sklearn.metrics import accuracy_score

#función que toma un conjunto de datos de entrada x
def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())

#Calcula la precisión comparando las etiquetas reales y_test con las etiquetas predichas y_pred
accuracy_score(y_test, y_pred.cpu().numpy())

0.9698

## Optimizadores y Funciones de pérdida

En el ejemplo anterior hemos calculado la función de pérdida y aplicado la regla de optimización de forma manual. Sin embargo, `Pytorch` nos ofrece funcionalidad que nos abstrae estos cálculos ofreciendo además flexibilidad para aplicar diferentes funciones de pérdida o algoritmos de optimización de manera sencilla. Podemos encontrar diferentes funciones de pérdida ya implementadas en el paquete `torch.nn`.

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

Mientras que los optimizadores se encuentran en el paquete `torch.optim`

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

Puedes ver la lista completa de funciones de pérdida y optimizadores disponibles en la [documentación](https://pytorch.org/docs/stable/index.html), aunque como ya has visto siempre puedes definir los tuyos propios fácilmente.

Una vez definidos estos dos objetos, nuestro bucle de entrenamiento se simplifica considerablemente.

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.66403
Epoch 20/100 Loss 1.09725
Epoch 30/100 Loss 0.88118
Epoch 40/100 Loss 0.72364
Epoch 50/100 Loss 0.62270
Epoch 60/100 Loss 0.55228
Epoch 70/100 Loss 0.50007
Epoch 80/100 Loss 0.45956
Epoch 90/100 Loss 0.42707
Epoch 100/100 Loss 0.40037


0.9683

## Modelos custom

Si bien en muchos casos definir una `red neuronal` como una secuencia de capas es suficiente, en otros casos será un factor limitante. Un ejemplo son las redes residuales, en las que no sólo utilizamos la salida de una capa para alimentar la siguiente si no que, además, le sumamos su propia entrada. Este tipo de arquitectura no puede ser definida con la clase `Sequential`, y para ello necesitamos hacer un modelo *customizado*. Para ello, `Pytroch` nos ofrece la siguiente sintaxis.

In [ ]:
# creamos una clase que hereda de `torch.nn.Module`

class ModeloPersonalizado(torch.nn.Module):

    # constructor
    def __init__(self, D_in, H, D_out):

        # llamamos al constructor de la clase madre
        super(ModeloPersonalizado, self).__init__()

        # definimos nuestras capas
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)

    # lógica para calcular las salidas de la red
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

En primer lugar, necesitamos definir una nueva clase que herede de la clase `torch.nn.Module`. Esta clase madre aportará toda la funcionalidad esencial que necesita una `red neuronal` (soporte GPU, iterar por sus parámeteros, etc). Luego, en esta clase necesitamos definir mínimos dos funciones:

- `init`: en el constructor llamaremos al constructor de la clase madre y después definiremos todas las capas que querramos usar en la red.
- `forward`: en esta función definimos toda la lógica que aplicaremos desde que recibimos los inputs hasta que devolvemos los outputs.

En el ejemplo anterior simplemente hemos replicado la misma red (puedes conseguir el mismo efecto usando la clase `Sequential`).

In [ ]:
model = ModeloPersonalizado(784, 100, 10)
# Codigo para saber si el modelo esta votando los datos en las cantidades correctas
x_prueba = torch.randn(64, 784)
print(x_prueba)
outputs = model(x_prueba)
outputs.shape

tensor([[-0.8344,  0.2306,  0.5819,  ..., -0.5459, -0.5584,  0.3841],
        [ 1.0140,  1.6143,  0.5310,  ..., -0.0141, -0.0879,  0.8197],
        [-1.6356,  1.6447, -0.4334,  ...,  0.0404,  0.1558, -0.0049],
        ...,
        [-0.4252,  0.2178,  1.2286,  ..., -0.4724, -0.1114, -0.1245],
        [ 0.4012,  0.1033, -0.3465,  ..., -0.0708, -1.3428, -1.5396],
        [ 1.5906,  1.3200,  1.0990,  ..., -0.8649,  0.6206,  0.1251]])


torch.Size([64, 10])

Ahora, podemos entrenar nuestra red de la misma forma que lo hemos hecho anteriormente.

In [ ]:
model.to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.60578
Epoch 20/100 Loss 1.05438
Epoch 30/100 Loss 0.85274
Epoch 40/100 Loss 0.70342
Epoch 50/100 Loss 0.60726
Epoch 60/100 Loss 0.53955
Epoch 70/100 Loss 0.48900
Epoch 80/100 Loss 0.44974
Epoch 90/100 Loss 0.41827
Epoch 100/100 Loss 0.39242


0.9692